In [16]:
import pandas as pd
import numpy as np
import os

gicc05 = pd.read_csv('/Users/quinnmackay/Documents/GitHub/BICC/Data Storage/Chronologies/GICC05_time_scale.tab', 
                     sep='\t', skiprows=41, usecols=[0,4, 6], names=['age', 'depth', 'sigma'])

gicc21 = pd.read_excel('/Users/quinnmackay/Documents/GitHub/BICC/Data Storage/Chronologies/GICC21.xlsx',
                        skiprows=1, sheet_name=1, usecols=[0,1,8], names=['age', 'sigma', 'depth'])

#3836

gicc05 = gicc05[gicc05['age']>3835]

df = pd.concat([gicc21, gicc05], ignore_index=True)

df.sort_values(by='age', ascending=True, inplace=True)

df.dropna(inplace=True)

df.reset_index(drop=True, inplace=True)

df


,age,sigma,depth
0,20,2.0,5.325
1,21,2.0,5.825
2,22,2.0,6.150
3,23,2.0,6.750
4,24,2.0,7.050
...,...,...,...
11679,11699,99.0,1624.080
11680,11700,99.0,1624.140
11681,11701,99.0,1624.170
11682,11702,99.0,1624.240


In [17]:
# do the calculation

# Define year intervals
min_age = df["age"].min()
max_age = df["age"].max()
intervals = np.arange(min_age, max_age, 100) #set year intervals, ignoring the end if less than 100 years

depths = np.interp(intervals, df["age"], df["depth"])

sigma_start = np.interp(intervals, df["age"], df["sigma"])
sigma_end = np.interp(intervals+99, df["age"], df["sigma"])

sigma_diff = sigma_end - sigma_start

#this is here to test for an error
for i in range(len(sigma_diff)):
    if sigma_diff[i] <= 0:
        sigma_diff[i] = 0.01

interval_df = pd.DataFrame({
    "depth_top": depths[:-1],
    "depth_bot": depths[1:],
    "duration": 100,
    "dur_unc": sigma_diff[:-1]
})

interval_df.dropna(inplace=True)

if os.name == 'nt':
    output_file = "E:/GitHub/BICC/Paleochrono/EDML-WDC_test/WDC/ice_age_intervals.txt"
elif os.name == 'posix':
    output_file = '/Users/quinnmackay/Desktop/temp/ice_age_intervals.txt'


interval_df.to_csv(output_file, sep="\t", index=False)

print(f"Done! Saved to: {output_file}")

Done! Saved to: /Users/quinnmackay/Desktop/temp/ice_age_intervals.txt
